# Aplicaciones para Libros

En esta investigación se realizará un análisis a distintos datsets de un start-up de libros a través de la herramienta SQL, con el fin de generar una propuesta de valor para un nuevo producto a través de los autores de los libros, la cantidad de libros, los editoriales, etc.


Por este motivo, se buscará lo siguiente:


1. Encuentra el número de libros publicados después del 1 de enero de 2000.
2. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
3. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
4. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
5. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

## Inicialización

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'practicum_student', # nombre de usuario
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # puerto de conexión
'db': 'data-analyst-final-project-db'} # nombre de la base de datos
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Examinar las tablas

In [3]:
query = """
SELECT 
     *
FROM 
    books
LIMIT 5;
"""
results = pd.io.sql.read_sql(query, con = engine)
results
    

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = """
SELECT 
     *
FROM 
    authors
LIMIT 5;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = """
SELECT 
     *
FROM 
    publishers
LIMIT 5;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = """
SELECT 
     *
FROM 
    ratings
LIMIT 5;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = """
SELECT 
     *
FROM 
    reviews
LIMIT 5;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Encuentra el número de libros publicados después del 1 de enero de 2000

In [8]:
query = """
SELECT 
     COUNT(*) AS count_books
FROM 
    books
WHERE
    CAST(publication_date AS date) > '2000-01-01';
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,count_books
0,819


### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [9]:
query = """
SELECT 
     books.book_id,
     books.title,
     COUNT(distinct review_id) as número_reseñas,
     AVG(ratings.rating) as rating_promedio
FROM 
    books

left join reviews
ON books.book_id = reviews.book_id

left join ratings
ON books.book_id = ratings.book_id

group by
    books.book_id,
    books.title
order by 
    rating_promedio;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,book_id,title,número_reseñas,rating_promedio
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [10]:
query = """
SELECT 
     books.publisher_id,
     publishers.publisher,
     COUNT(books.book_id) as count_books
    
FROM 
    books

left join publishers
ON books.publisher_id = publishers.publisher_id

left join ratings
ON books.book_id = ratings.book_id

WHERE
    num_pages > 50
group by
    books.publisher_id,
    publishers.publisher_id
order by
    count_books desc
LIMIT 1;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,publisher_id,publisher,count_books
0,212,Penguin Books,374


###  Identifica al autor que tiene la más alta calificación promedio del libro

In [18]:
query = """
SELECT authors.author, AVG(ratings.rating) AS avg_rating
FROM authors 
JOIN books ON authors.author_id = books.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating_id) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,author,avg_rating
0,Diana Gabaldon,4.3


###  Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [15]:
query = """
SELECT AVG(review_count) AS avg_review_count
FROM (
    SELECT username, COUNT(review_id) AS review_count
    FROM reviews 
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(DISTINCT book_id) > 50
    )
    GROUP BY username
) AS subquery;
"""
results = pd.io.sql.read_sql(query, con = engine)
results

,avg_review_count
0,24.333333


## Conclusiones Generales

Después de analizar los distintos dataset de la editorial en SQL se obtuvo lo siguiente: 

1. 819 libros fueron publicados después del 1 de enero de 2000

2. El número de reseñas de los libros van desde 2 reseñas a 3, y la calificación promedio va desde 1.5 hasta los 5 puntos.

3. La editorial con más libros de más de 60 páginas es Penguin Books con 374 libros

4. El autor con la calificación promedio más alta es Diana Gabaldon con 4.3 de calificación promedio.

5. El número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libras es apróximadamente 25 reseñas.